Logistic Regression
-------------------

This example performs logistic regression. The corresponding jupyter notebook is found [here](https://github.com/NervanaSystems/ngraph/blob/master/examples/walk_through/Logistic_Regression_Part_1.ipynb).
We want to classify an observation $x$ into one of two classes, denoted by $y=0$ and $y=1$. Using a simple linear model:
$$\hat{y}=\sigma(Wx)$$

we want to find the optimal values for $W$. Here, we use gradient descent with **a learning rate of $\alpha$** and the cross-entropy as the error function.

### Axes

The nervana graph uses `Axes` to attach shape information to tensors. The identity of `Axis` objects are used to pair and specify dimensions in symbolic expressions. The function ``ng.make_axis`` will create an ``Axis`` object with an optionally supplied `name` argument. For example:

In [ ]:
import ngraph as ng
import ngraph.transformers as ngt
    
my_axis = ng.make_axis(length=256, name='my_axis')

Alternatively, we can use a ``NameScope`` to set the names of the various axes. A ``NameScope`` is an object that sets the name of an object to that of its assigned attribute. So when we set ``ax.N`` to an ``Axis`` object, the ``name`` of the object is automatically set to ``ax.N``. This a convenient way to define axes, so we use this approach for the rest of this example.

In [ ]:
ax = ng.make_name_scope("ax")
ax.N = ng.make_axis(length=128, batch=True)
ax.C = ng.make_axis(length=4)

We add ``batch`` as a property to ``ax.N`` to indicate that the axis is a batch axis. **A batch axis is held out of the default set of axes reduced in reduction operations such as sums.**

### Building the graph
Our model has three placeholders: ``X``, ``Y``, and ``alpha``, each of which need to have axes defined. ``alpha`` is a scalar, so we pass in empty axes:

In [ ]:
alpha = ng.placeholder(axes=())

``X`` and ``Y`` are tensors for the input and output data, respectively. **Our convention is to use the last axis for samples**.  The placeholders can be specified as:

In [ ]:
X = ng.placeholder(axes=[ax.C, ax.N])
Y = ng.placeholder(axes=[ax.N])

We also need to specify the training weights, ``W``.  **Unlike a placeholder, ``W`` should retain its value from computation to computation (for example, across mini-batches of training).**  Following TensorFlow, we call this a *variable*.  We specify the variable with both ``Axes`` and also an initial value:

In [ ]:
W = ng.variable(axes=[ax.C - 1], initial_value=0)

The nervana graph axes are agnostic to data layout on the compute device, so the ordering of the axes does not matter. As a consequence, **when two tensors are provided to a `ng.dot()` operation**, for example, ***one needs to indicate which are the corresponding axes that should be matched together.*** We use "dual offsets" of +/- 1 to mark which axes should be matched during a multi-axis operation, which gives rise to the `ax.C - 1` observed above. Every axis is a member of a family of axes we call duals of the axis, and each axis in the family has a position. ***When you create an axis, its dual position is 0***. When ``ng.dot`` is called, **it pairs axes in the first and second arguments** that are **_of the same dual family_** and have **consecutive positions**. For more information, see the `Axes` section of the user guide [here](https://ngraph.nervanasys.com/docs/latest/axes.html)

Now we can estimate ``y`` as ``Y_hat`` and compute the average loss ``L``:

In [ ]:
Y_hat = ng.sigmoid(ng.dot(W, X))
L = ng.cross_entropy_binary(Y_hat, Y, out_axes=()) / ng.batch_size(Y_hat)

Here we use several ngraph functions, including ``ng.dot`` and ``ng.sigmoid``. Since a tensor can have multiple axes, we need a way to mark which axes in the first argument of ``ng.dot`` are to act on which axes in the second argument. Please also note that the `W` has been defined with one axis, while `X` has two axis. Every tensor component along C axis in `X` is being dot-producted with `W`, and the `N` results are stored in `Y_hat`, that has only one axis, the `N` axis.

As already noted, when ``ng.dot`` is called, **it pairs axes in the first and second arguments that are of the same dual family** and have **consecutive positions**. In what shown above, the dual position of the `X` axis of `W` was "decreased" to `-1`, being `0` the default position assigned when `W` was created. Remember that we want the variable `W` to act on the `ax.C` axis of the input `X`, so we want the axis for `W` to be in the position before `ax.C`, which we can obtain with `ax.C - 1`. 

Once `Y_hat` has been computed (the whole batch computation was defined above), we can move on and correct the weights in `W`. Gradient descent requires computing the gradient, $\frac{dL}{dW}$


In [ ]:
grad = ng.deriv(L, W)

The ``ng.deriv`` function computes the backprop using **autodiff**. We are almost done as we are now ready to update ``W``.  The update step (which is an Op that will be carried out at the time of real computation on the device) computes the new weight and assigns it to ``W``:

In [ ]:
update = ng.assign(W, W - alpha * grad / ng.tensor_size(Y_hat))
type(update)

ngraph.op_graph.op_graph.AssignOp

### Computation

**Now we create a transformer and define a computation for learning**. In order to do so, we pass the ops from which we want to retrieve the results for, followed by the placeholders:


In [ ]:
transformer = ngt.make_transformer()
update_fun = transformer.computation([L, W, update], alpha, X, Y)

Here, the **computation will return three values** for the** ``L``, ``W``, and ``update``**, given inputs to fill the placeholders,** $\alpha$ (LR), X (inputs), Y (expected outputs).**

Therefore to run the computation we need to generate input data. Here below the input data, ``X`` and ``Y``, is synthetically generated as a mixture of two Gaussian distributions in 4-d space.  We shape our entire dataset as 10 mini-batches of 128 samples each, which we create with a convenient function:

In [ ]:
import gendata

g = gendata.MixtureGenerator([.5, .5], (ax.C.length,))
XS, YS = g.gen_data(ax.N.length, 10)


Finally, we train the model across the 10 epochs, printing the loss and updated weights. Please note that we are using a decreasing policy (with the epoch number) for $\alpha$. Also note that there is no need to invoke update_fun specifying the outputs, as they were specified at definition time. Now we need only to feed the inputs into ``update_fun``

In [ ]:
    for i in range(10):
        for xs, ys in zip(XS, YS):
            loss_val, w_val, _ = update_fun(5.0 / (1 + i), xs, ys)
            print("W: %s, loss %s" % (w_val, loss_val))

W: [ 2.26889729  5.24611712  0.55897683  1.15695345], loss 0.174076288939
W: [ 2.31940055  5.31047344  0.5742268   1.17916524], loss 0.135152488947
W: [ 2.36883974  5.38630724  0.53643304  1.14578247], loss 0.162494510412
W: [ 2.38650537  5.43609238  0.52788138  1.16376483], loss 0.186021730304
W: [ 2.41335869  5.49430609  0.49320835  1.23979235], loss 0.166057318449
W: [ 2.45394778  5.54222155  0.47551811  1.21743679], loss 0.154901966453
W: [ 2.48280835  5.52348232  0.3891854   1.28643167], loss 0.244514852762
W: [ 2.5384531   5.54951572  0.42466164  1.42140508], loss 0.172828257084
W: [ 2.55841827  5.58289671  0.38803348  1.44254649], loss 0.197061210871
W: [ 2.63447213  5.58814955  0.41302529  1.52762759], loss 0.19623811543
W: [ 2.63638568  5.62076712  0.40745494  1.49877357], loss 0.170830905437
W: [ 2.65651441  5.65096331  0.41057384  1.49799263], loss 0.12679502368
W: [ 2.67850089  5.68695545  0.38931185  1.47049344], loss 0.155834704638
W: [ 2.68363738  5.71074533  0.38148576 

Also see Part 2 of logistic regressions, which walks uses through adding additional variables, computations, and dimensions. <br>  

